In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
train_df=pd.read_csv('../input/titanic/train.csv')
train_df.head()

In [71]:
test_df=pd.read_csv('../input/titanic/test.csv')
test_df.head()

In [72]:
train_df.shape 

In [73]:
test_df.shape

In [74]:
train_df.info()

In [75]:
test_df.info()

In [76]:
train_df.describe()

In [77]:
test_df.describe()


In [78]:
Cabin_Text=[]
for i in train_df['Cabin']:
    Cabin_Text.append(str(i)[0])
        
train_df['Cabin_Text']=Cabin_Text

train_df['Cabin_Text'].replace({'n': np.nan},inplace=True)

train_df.head()

In [79]:
Cabin_Text=[]
for i in test_df['Cabin']:
    Cabin_Text.append(str(i)[0])
        
test_df['Cabin_Text']=Cabin_Text

test_df['Cabin_Text'].replace({'n': np.nan},inplace=True)

test_df.head()

In [80]:
train_df.isna().sum()


In [81]:
test_df.isna().sum()


In [82]:
cond=train_df['Survived']==0

train_df['Age'].fillna(cond.map({True:train_df.loc[train_df['Survived']==0]['Age'].median(),
                                False:train_df.loc[train_df['Survived']==1]['Age'].median()
                                }),inplace=True)

test_df['Age'].fillna(test_df['Age'].median(),inplace=True)

train_df['Cabin_Text'].fillna('C',inplace=True)

test_df['Cabin_Text'].fillna('C',inplace=True)

train_df['Embarked'].fillna('S',inplace=True)

test_df['Fare'].fillna(test_df['Fare'].median(),inplace=True);


In [83]:
#Since there are no null values in passenger ID we can directly use it as default ID
#setting PassengerId as the Id
train_df.set_index('PassengerId',inplace=True)
test_df.set_index('PassengerId',inplace=True)

In [84]:
train_df=train_df.drop("Name",axis=1)


In [85]:
train_df=train_df.drop("Ticket",axis=1)


In [86]:
train_df=train_df.drop("Cabin",axis=1)

In [87]:
test_df=test_df.drop("Name",axis=1)

In [88]:
test_df=test_df.drop("Ticket",axis=1)

In [89]:
test_df=test_df.drop("Cabin",axis=1)

In [90]:
cat_attributes= ['Sex','Cabin_Text','Embarked']
for i in cat_attributes:
    le=LabelEncoder()
    arr=np.concatenate((train_df[i], test_df[i])).astype(str)
    le.fit(arr)
    train_df[i]=le.transform(train_df[i].astype(str))
    test_df[i]=le.transform(test_df[i].astype(str))

In [91]:
train_df['Survived']=train_df['Survived'].replace({True:1,False:0})


In [92]:
X=train_df.drop('Survived',axis=1)
y=train_df['Survived']

In [93]:
X.columns


In [94]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [95]:
from sklearn.ensemble import RandomForestClassifier
random=RandomForestClassifier()
random.fit(X_train,y_train)

In [96]:
y_pred=random.predict(X_val)

In [97]:
score=random.score(X_val,y_val)
print(score)

In [98]:
mae_random = metrics.mean_absolute_error(y_val, y_pred)
print(mae_random)

In [99]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_val, y_pred) 
print(cm)

In [100]:
y_pred
m=0
n=0
for i in y_pred:
    if i==0:
        m+=1
    else:
        n+=1
print(m,'people have survied.')
print(n,'people have died.')

In [101]:
test_df


In [102]:
y_pred=random.predict(test_df)
submission=pd.DataFrame({'Survived':y_pred},index=test_df.index)
submission.head()

In [103]:
y_pred
m=0
n=0
for i in y_pred:
    if i==0:
        m+=1
    else:
        n+=1
print(m,'people have survied.')
print(n,'people have died.')

In [104]:
submission.to_csv('submission.csv')
